#### Mie comparison notebook
This notebook compares Mie Mueller matrix calculations done in Mitsuba versus a reference [Mishchenko et al. 2012].  
Run all cells and use the sliders to adjust parameters to the Mie calculations.  
Written by Kate 05/14/21.

In [1]:
import os
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import widgets, interact

In [2]:
# Read in file data
mueller_files = [ f for f in os.listdir('mie_data_mono_script/') if f.endswith('.npy') ]
mueller_data = [ np.load('mie_data_mono_script/' + f) for f in mueller_files ]

mueller_dict = {}

for i, f in enumerate(mueller_files):
    mueller_dict[f] = mueller_data[i]
    
thetas = mueller_data[0][:,0]

In [3]:
# Set up UI
wav_text = "Wavelength (nm)"
wav_slider = widgets.FloatSlider(value = 600.0, min = 400.0, max = 800.0, step = 100.0, continuous_update = False)
wav_label = widgets.Label(value = wav_text, layout = widgets.Layout(width = '250px'))
wav_box = widgets.HBox([wav_label, wav_slider])

rad_text = "Radius (nm)"
rad_slider = widgets.SelectionSlider(options = [400.0, 500.0, 600.0, 700.0, 800.0, 900.0, 1000.0, 5000.0, 10000.0],
    value = 1000.0, disabled = False, continuous_update = False, orientation = 'horizontal', readout = True)
rad_label = widgets.Label(value = rad_text, layout = widgets.Layout(width = '250px'))
rad_box = widgets.HBox([rad_label, rad_slider])

ior_imag_text = "IOR sphere (imag)"
ior_imag_slider = widgets.SelectionSlider(options=[0.0, 1e-2, 1e-1, 5e-1, 1e0],
    value = 0.0, disabled = False, continuous_update = False, orientation = 'horizontal', readout = True)
ior_imag_label = widgets.Label(value = ior_imag_text, layout = widgets.Layout(width = '250px'))
ior_imag_box = widgets.HBox([ior_imag_label, ior_imag_slider])

warning_text = "There is no data for this set of parameters."
warning_label = widgets.Button(description = warning_text, button_style = 'warning', 
                               layout = widgets.Layout(width = '300px'))

ui = widgets.VBox([wav_box, rad_box])

# Show warning about missing data
def warning_on_display():
    return len(ui.children) == 3

# Format float for filename
def float_to_str(f):
    f_str = '{:.2e}'.format(f)
    f_str = f_str.replace('e', 'D')
    f_str = f_str.replace('.', '_')
    return f_str

# Convert values to filename
def values_to_filenames(w, r, i):
    w_str = float_to_str(w / 1000.0) # nm to um
    r_str = float_to_str(r / 1000.0) # nm to um
    i_str = float_to_str(i)
    
    file_pfx = 'w' + w_str + 'r' + r_str + 'i' + i_str
    return file_pfx + '.npy', file_pfx + '_ours.npy'

# Make 2x2 figure
fig = make_subplots(rows = 2, cols = 2, subplot_titles = ['m11', 'm12', 'm33', 'm34'])

# Fill in figure with line plots
for i in range(2):
    for j in range(2):
        k = i*2 + j
        
        # Reference data (from Mishchenko Fortran code)
        fig.add_trace(go.Scatter(
            x = thetas,
            y = np.zeros(len(thetas)),
            mode = 'lines',
            name = 'Mishchenko',
            legendgroup = 'Mishchenko',
            line = dict(
                        width = 1,
                        color = "blue"
                   ),
            showlegend = (i == 0 and j == 0)
            ),
            row = i+1,
            col = j+1
        )

        # Our data
        fig.add_trace(go.Scatter(
            x = thetas,
            y = np.zeros(len(thetas)),
            mode = 'lines',
            name = 'Ours',
            legendgroup = 'Ours',
            line = dict(
                        width = 1,
                        color = "red"
                   ),
            showlegend = (i == 0 and j == 0)
            ),
            row = i+1,
            col = j+1
        )
        
fig.update_layout(
    title = "Mie Mueller matrix elements",
    xaxis_title = "theta (deg)",
    yaxis_title = "value",
    showlegend = True,
    legend = dict(
        orientation = 'h',
        xanchor = "center",
        yanchor = "top",
        y=-0.1,
        x=0.5
    )
)

fig.update_yaxes(
    type = 'log', 
    row = 1,
    col = 1
)

fig_widget = go.FigureWidget(fig)

# Update data in plots according to slider values
def update(wav_slider, rad_slider, ior_imag_slider):
    file_ref, file_ours = values_to_filenames(wav_slider, rad_slider, ior_imag_slider)
    
    if file_ref in mueller_dict and file_ours in mueller_dict:
        data_ref = mueller_dict[file_ref]
        data_ours = mueller_dict[file_ours]

        # Erase warning message
        if warning_on_display():
            ui.children = ui.children[:-1]
    else:
        # Show warning message
        if not warning_on_display():
            ui.children = (*ui.children, warning_label)
        return
    
    with fig_widget.batch_update():
        fig_widget.data[0].y = data_ref[:,1]
        fig_widget.data[1].y = data_ours[:,1]
        fig_widget.data[2].y = data_ref[:,2]
        fig_widget.data[3].y = data_ours[:,2]
        fig_widget.data[4].y = data_ref[:,3]
        fig_widget.data[5].y = data_ours[:,3]
        fig_widget.data[6].y = data_ref[:,4]
        fig_widget.data[7].y = data_ours[:,4]

out = widgets.interactive_output(update, {'wav_slider': wav_slider, 'rad_slider': rad_slider, 'ior_imag_slider': ior_imag_slider})
display(ui)
fig_widget

FigureWidget({
    'data': [{'legendgroup': 'Mishchenko',
              'line': {'color': 'blue', 'width': 1},…